>####
>####  
>####有加密列的全量banda_etl
=================?
>#####
>#####  
>#####
>#####
>#####
>##

In [1]:
import sys
#notification导表到notification_etl_s3
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
import json
import pytz
import hmac
import pyspark.sql.functions as F
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
tablemap={"banda_stream_etl":"banda"} #,
# ,"banda_stream_etl":"1"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1   AND a.kind <> 'delete' "
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .enableHiveSupport()\
    .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    print(json_content)
    return json_content
colmap= get_secret_obj()
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None:
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
    return ""
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    secret_Col=[]
    if colmap.get(dbType)!=None and colmap[dbType]!=None and colmap[dbType].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
    return sh256Col.format(hmac_key=hmac_key)
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
    if tableName=='t_call_center_info' :
        tablecolum="desc " +databaseName+"."+tableName;
        tableSchema= spark.sql(tablecolum).collect()
        tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
        spark.catalog.refreshTable(databaseName+"."+tableName)
        dbtype=tablemap[databaseName]
        secret_sql=add_secret_col(dbtype,tableName,getTableColum(tableSchema))+", year"
        sql="select "+secret_sql+ tablesql_1+databaseName+"."+tableName+tablesql_2
        df=spark.sql(sql).drop("etlindex")
        if colmap.get(dbtype) and colmap[dbtype].get(tableName)!=None :
            for i in colmap[tablemap[databaseName]][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
            colmap["extraColumn"][dbtype][tableName].insert(0,"*")
            df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
            df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
    #     df.drop("col_x").show()
        df.drop("col_x").write.mode("overwrite").partitionBy("year").orc(tablepath)
# print(__name__)
if __name__ == "builtins":
#     注册udf
    spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
#     spark.conf.set("spark.sql.shuffle.partitions",10000000)
#     spark.conf.set("spark.sql.files.maxPartitionBytes",20)
    for databaseName in tablemap:
        print(databaseName) 
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1629793254659_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'hmac_key': 'xUIAZ3grtPOxaPNK', 'mobileType': ['phone', 'phone_number', 'user_mobile', 'number', 'caller', 'callee', 'to', 'mobile_phone', 'mobile', 'backup_mobile'], 'extraColumn': {'banda': {'t_personal_info': ['cast(substring(credential_no_x, 11, 2) AS bigint) as birth_year', 'CASE WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900) ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000) END as age ']}}, 'luzon': {'t_auto_review_loan': ['ktp', 'mobile'], 't_bankcard': ['card_no'], 't_contact': ['mobile', 'credential_no'], 't_customer': ['mobile', 'imei'], 't_loan_app': ['credential_no'], 't_personal_info': ['credential_no', 'email'], 't_record_contact': ['mobile'], 't_record_personal_info': ['credential_no']}, 'ddb_event_track_s3': {'id_adapundi_event_track': ['nativePhoneNum', 'nativePhoneNum2', 'lineNumber']}, 'banda': {'t_admin': ['full_name'], 't_auto_re

>>>>>>
>>>>>>
>>>>>>
>>>>>>
step function的代碼

>>>>>>
>>>>>>

In [7]:
# -*- coding: UTF-8 -*-
import sys
#notification导表到notification_etl_s3
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
import json
import pytz
import hmac
import pyspark.sql.functions as F
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
# tablemap={"banda_stream_etl":"banda"} #,
tablemap="#tableMapParams#"
# tableList={"banda":["*"]} 
tableList="#tableListParams#"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1   AND a.kind <> 'delete' "
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .enableHiveSupport()\
    .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None:
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
    return ""
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    secret_Col=[]
    if colmap.get(dbType)!=None and colmap[dbType]!=None and colmap[dbType].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
    return sh256Col.format(hmac_key=hmac_key)
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
    tablecolum="desc " +databaseName+"."+tableName;
    tableSchema= spark.sql(tablecolum).collect()
    tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
    spark.catalog.refreshTable(databaseName+"."+tableName)
    dbtype=tablemap[databaseName]
    secret_sql=add_secret_col(dbtype,tableName,getTableColum(tableSchema))+", year"
    sql="select "+secret_sql+ tablesql_1+databaseName+"."+tableName+tablesql_2
    df=spark.sql(sql).drop("etlindex")
    if colmap.get(dbtype) and colmap[dbtype].get(tableName)!=None :
        for i in colmap[tablemap[databaseName]][tableName]:
            df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
    if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
        colmap["extraColumn"][dbtype][tableName].insert(0,"*")
        df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
        df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
#     df.drop("col_x").show()
    df.drop("col_x").write.mode("overwrite").partitionBy("year").orc(tablepath)
print(__name__)
if __name__ == "__main__":
#     注册udf
    spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        print(databaseName)
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        if tablemap[databaseName] in tableList :
            ls=[]
            for table in tableList[tablemap[databaseName]]:
                if table=='*':
                    ls =tablelist
                else:
                    ls=ls+[tb for tb in tablelist if tb["tableName"]==table]
            print(ls)
            executor=None
            with futures.ThreadPoolExecutor(max_workers=10) as executor:
                futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
                for  future in futures_result[0]:
                    print(future.result())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

builtins
<function hmac_sha256 at 0x7fa3b84279e0>
banda_stream_etl
[Row(database='banda_stream_etl', tableName='t_admin', isTemporary=False), Row(database='banda_stream_etl', tableName='t_admin_audit', isTemporary=False), Row(database='banda_stream_etl', tableName='t_admin_group_rel', isTemporary=False), Row(database='banda_stream_etl', tableName='t_af_bill', isTemporary=False), Row(database='banda_stream_etl', tableName='t_ai_rudder_schedule_mobile', isTemporary=False), Row(database='banda_stream_etl', tableName='t_auto_review_loan', isTemporary=False), Row(database='banda_stream_etl', tableName='t_bankcard', isTemporary=False), Row(database='banda_stream_etl', tableName='t_bankcard_merchant', isTemporary=False), Row(database='banda_stream_etl', tableName='t_bankcard_verify', isTemporary=False), Row(database='banda_stream_etl', tableName='t_case_distribution_log', isTemporary=False), Row(database='banda_stream_etl', tableName='t_channel_details', isTemporary=False), Row(database='band

>####
>####  
>####有加密列的增量banda测试
=================?
>#####
>#####  

>#####
>#####
>##

In [34]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime,timedelta
import copy
import json
import pytz
import hmac
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import types as T
dbmap={
    "banda":{
        "increment_database":"banda_stream_etl",
        "database":"banda-etl-s3",
        "realTime_path":"banda"
    }
}
nowdate=(datetime.now()).strftime('%Y-%m-%d')
yesterday=(datetime.now()+ timedelta(-1)).strftime('%Y-%m-%d')
sqltemp="""with {update_snapshot_table} as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
      FROM `{increment_database}`.`{tableNm}`
      where date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
      )
    where row_num = 1
  )
select  {select_sql} 
from `{database}`.`{tableNm}` l
left join {update_snapshot_table}
on l.id = {update_snapshot_table}.id
where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete'
union
select {update_snapshot_sql} from (
  select *
  from `{increment_database}`.`{tableNm}`
  where
    date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
    and kind = 'insert'
  ) new
left join {update_snapshot_table}
on new.id = {update_snapshot_table}.id where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete' """
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()   
hmac_key=colmap["hmac_key"]
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None and value!='':
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b,update_snapshot_table,dbtype,tableNm):
    colum=""
    snapshot_colum=""
    sqlstrtemp="if({update_snapshot_table}.id is null, {colNm}     , {snapshot_colNm})  {ascolNm} ,"
    sqlstr_secret_temp=" if({update_snapshot_table}.id is null, {colNm}     , hmac_sha256('{hmac_key}','{ascolNm}',{snapshot_colNm})) {ascolNm}_x , "
    for index in range(len(b)):
        if(index==1 or (index>2 and index<len(b)-7)):
#             print(index,b[index])
#             在year前面加上加密列
            if(index==len(b)-8) and colmap[dbtype]!=None and colmap[dbtype].get(tableNm)!=None:
                for secret_col in colmap[dbtype][tableNm]:
                    sqlstr=copy.copy(sqlstr_secret_temp)
                    colum=colum+sqlstr.format(colNm=setDef("string",secret_col,False),snapshot_colNm=setDef("string",secret_col,True),ascolNm=secret_col,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
                    snapshot_colum=snapshot_colum+setsecret_SnapshotDef(secret_col).format(update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)+" ,"
            sqlstr=copy.copy(sqlstrtemp)
            colum=colum+sqlstr.format(colNm=isX(dbtype,tableNm,setDef(b[index]["data_type"],b[index]["col_name"],False)),snapshot_colNm=setDef(b[index]["data_type"],b[index]["col_name"],True),ascolNm=b[index]["col_name"],update_snapshot_table=update_snapshot_table)
            snapshot_colum=snapshot_colum+setSnapshotDef(b[index]["data_type"],b[index]["col_name"]).format(update_snapshot_table=update_snapshot_table)+" ,"
    return colum[0:len(colum)-2],snapshot_colum[0:len(snapshot_colum)-2]
def isX(type,table,column):
    columnList=column.split(".")
    if len(columnList)>0 and  colmap.get(type)!=None and colmap[type].get(table)!=None and columnList[len(columnList)-1] in colmap[type][table]:
        return column+"_x"
    return column
def setDef(type,table_col,is_snapshot):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        if is_snapshot:
            return "ifnull({update_snapshot_table}."+table_col_x+",0)  "
        else:
            return "ifnull(l."+table_col_x+",0) "
    else:
        if is_snapshot:
            return "{update_snapshot_table}."+table_col_x
        else:
            return "l."+table_col_x
def setSnapshotDef(type,table_col):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        return "ifnull({update_snapshot_table}."+table_col_x+",0) "+table_col
    else:
        return "{update_snapshot_table}."+table_col_x
# def setDef(type,table_col,is_snapshot):
#     if is_snapshot:
#         return "{update_snapshot_table}."+table_col
#     else:
#         return "l."+table_col
def setsecret_SnapshotDef(table_col):
    table_col_x=copy.copy(table_col)
#     print("hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x")
    return "hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x"

def execute(tablerow,dbtype):
#     print(tablerow,dbtype)
    tableName=tablerow["tableName"]
#      tableName=='t_engine_rule_detail' or 
    if tableName=='t_loan_issue':
        update_snapshot_table="update_snapshot_"+dbtype+"_"+tableName
        spark.catalog.refreshTable("`"+dbmap[dbtype]["database"]+"`."+tableName)
        col=spark.sql("desc `"+dbmap[dbtype]["increment_database"]+"`."+tableName).collect()
        #     #真正的列,增量的列
        real_col,snapshot_col=getTableColum(col,update_snapshot_table,dbtype,tableName)
        sql=copy.copy(sqltemp)
        real_sql=sql.format(increment_database=dbmap[dbtype]["increment_database"],tableNm=tableName,nowdate=nowdate,select_sql=real_col.format(update_snapshot_table=update_snapshot_table),update_snapshot_sql=snapshot_col.format(update_snapshot_table=update_snapshot_table),database=dbmap[dbtype]["database"],yesterday=yesterday,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
        df=spark.sql(real_sql).drop("etlindex")
        if colmap[dbtype].get(tableName)!=None :
            for i in colmap[dbtype][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
            colmap["extraColumn"][dbtype][tableName].insert(0,"*")
            df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
            df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
#         df.show()
        df.drop("col_x").write.option("path", "s3://rupiahplus-data-warehouse/stream/etl_s3_temp/"+dbtype+"_"+tableName).mode("overwrite").partitionBy("year").format("orc").saveAsTable("etl_s3_temp."+dbtype+"_"+tableName);
        tablepath='s3://rupiahplus-data-warehouse/aliyun/'+dbmap[dbtype]["realTime_path"]+'/'+tableName+"/"
        spark.sql("select  * from  "+"etl_s3_temp."+dbtype+"_"+tableName).write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
    print("start-----",datetime.now())
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport()\
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode","nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    spark.conf.set("spark.files.overwrite","true")
#     ,Apache Arrow：一个跨平台的在内存中以列式存储的数据层，用来加速大数据分析速度。其可以一次性传入更大块的数据，pyspark中已经有载入该模块，需要打开该设置：
    spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#     创建表时自动删除已存在的目录
    spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
    for dbtype in dbmap:
        databasesql="show tables in "+dbmap[dbtype]["increment_database"]
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
    #         for row in tablelist:
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(execute, table,dbtype) for table in tablelist])
            for  future in futures_result[0]:
                print(future.result())
                print("end-----",datetime.now())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start----- 2021-03-25 08:09:22.707359
t_admin
t_admin_audit
t_admin_group
t_admin_group_rel
t_af_bill
t_ai_rudder_schedule_mobile
t_amount_reduce_record
t_auto_review_loan
t_bankcard
t_bankcard_merchant
t_bankcard_verify
t_call_center_log
t_case_distribution_log
t_channel_details
t_clear_detail_log
t_code
t_collection_audit
t_collection_blacklist
t_contact
t_coupon_request
t_coupon_send
t_customer
t_customer_app
t_customer_device_info
t_customer_install_info
t_customer_tag_rel
t_department_group
t_deposit_callback_log
t_employment
t_engine_rule_detail
t_file
t_group
t_hit_blacklist_record
t_invitation
t_job_metric
t_loan_app
t_loan_app_review_summary
t_loan_app_status_log
t_loan_issue
t_loan_log
t_loan_tag_rel
t_login_log
t_lpay
t_lpay_deposit
t_message
t_mobile_tag_rel
t_personal_info
t_record_bankcard
t_record_contact
t_record_employment
t_record_file
t_record_personal_info
t_reduce
t_review_blacklist
t_review_step_execution
t_send_coupon_rel
t_sms
t_test_customer
t_thirdparty_data
t

>####
>####  
>####有加密列的增量banda正式

=================?
>#####
>#####  

>#####
>#####
>##

In [4]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime,timedelta
import copy
import json
import pytz
import hmac
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import types as T
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("spark.driver.maxResultSize", "4g") \
    .enableHiveSupport()\
    .getOrCreate()
spark.conf.set("hive.exec.dynamic.partition.mode","nonstrict");
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.shuffle.partitions",20)
spark.conf.set("spark.files.overwrite","true")
#     ,Apache Arrow：一个跨平台的在内存中以列式存储的数据层，用来加速大数据分析速度。其可以一次性传入更大块的数据，pyspark中已经有载入该模块，需要打开该设置：
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#     创建表时自动删除已存在的目录
dbmap={
    "banda":{
        "increment_database":"banda_stream_etl",
        "database":"banda-etl-s3",
        "realTime_path":"banda"
    }
}
nowdate=(datetime.now()).strftime('%Y-%m-%d')
yesterday=(datetime.now()+ timedelta(-1)).strftime('%Y-%m-%d')
sqltemp="""with {update_snapshot_table} as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `{increment_database}`.`{tableNm}`
         where date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
      )
    where row_num = 1
  )
select  {select_sql} 
from `{database}`.`{tableNm}` l
left join {update_snapshot_table}
on l.id = {update_snapshot_table}.id
where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete'
union
select {update_snapshot_sql} from (
  select *
  from `{increment_database}`.`{tableNm}`
  where
    date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
    and kind = 'insert'
  ) new
left join {update_snapshot_table}
on new.id = {update_snapshot_table}.id where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete' """
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()   
hmac_key=colmap["hmac_key"]
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None and value!='':
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b,update_snapshot_table,dbtype,tableNm):
    colum=""
    snapshot_colum=""
    sqlstrtemp="if({update_snapshot_table}.id is null, {colNm}     , {snapshot_colNm})  {ascolNm} ,"
    sqlstr_secret_temp=" if({update_snapshot_table}.id is null, {colNm}     , hmac_sha256('{hmac_key}','{ascolNm}',{snapshot_colNm})) {ascolNm}_x , "
    for index in range(len(b)):
        if(index==1 or (index>2 and index<len(b)-7)):
#             print(index,b[index])
#             在year前面加上加密列
            if(index==len(b)-8)and  colmap.get(dbtype)!=None and colmap[dbtype]!=None and colmap[dbtype].get(tableNm)!=None:
                for secret_col in colmap[dbtype][tableNm]:
                    sqlstr=copy.copy(sqlstr_secret_temp)
                    colum=colum+sqlstr.format(colNm=setDef("string",secret_col,False),snapshot_colNm=setDef("string",secret_col,True),ascolNm=secret_col,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
                    snapshot_colum=snapshot_colum+setsecret_SnapshotDef(secret_col).format(update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)+" ,"
            sqlstr=copy.copy(sqlstrtemp)
            colum=colum+sqlstr.format(colNm=isX(dbtype,tableNm,setDef(b[index]["data_type"],b[index]["col_name"],False)),snapshot_colNm=setDef(b[index]["data_type"],b[index]["col_name"],True),ascolNm=b[index]["col_name"],update_snapshot_table=update_snapshot_table)
            snapshot_colum=snapshot_colum+setSnapshotDef(b[index]["data_type"],b[index]["col_name"]).format(update_snapshot_table=update_snapshot_table)+" ,"
    return colum[0:len(colum)-2],snapshot_colum[0:len(snapshot_colum)-2]
def isX(type,table,column):
    columnList=column.split(".")
    if len(columnList)>0 and  colmap.get(type)!=None and colmap[type].get(table)!=None and columnList[len(columnList)-1] in colmap[type][table]:
        return column+"_x"
    return column
def setDef(type,table_col,is_snapshot):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        if is_snapshot:
            return "ifnull({update_snapshot_table}."+table_col_x+",0)  "
        else:
            return "ifnull(l."+table_col_x+",0) "
    else:
        if is_snapshot:
            return "{update_snapshot_table}."+table_col_x
        else:
            return "l."+table_col_x
def setSnapshotDef(type,table_col):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        return "ifnull({update_snapshot_table}."+table_col_x+",0) "+table_col
    else:
        return "{update_snapshot_table}."+table_col_x
def setsecret_SnapshotDef(table_col):
    table_col_x=copy.copy(table_col)
#     print("hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x")
    return "hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x"
#     return "sha2({update_snapshot_table}."+table_col+",256) " +table_col+"_x"
def execute(tablerow,dbtype):
#     print(tablerow,dbtype)
    tableName=tablerow["tableName"]
    if tableName=='t_auto_review_loan':
        update_snapshot_table="update_snapshot_"+dbtype+"_"+tableName
        print("`"+dbmap[dbtype]["database"]+"`."+tableName)
        print("`"+dbmap[dbtype]["increment_database"]+"`."+tableName)
        spark.catalog.refreshTable("`"+dbmap[dbtype]["database"]+"`."+tableName)
        spark.catalog.refreshTable("`"+dbmap[dbtype]["increment_database"]+"`."+tableName)
        col=spark.sql("desc `"+dbmap[dbtype]["increment_database"]+"`."+tableName).collect()
        #     #真正的列,增量的列
        real_col,snapshot_col=getTableColum(col,update_snapshot_table,dbtype,tableName)
        sql=copy.copy(sqltemp)
        real_sql=sql.format(increment_database=dbmap[dbtype]["increment_database"],tableNm=tableName,nowdate=nowdate,select_sql=real_col.format(update_snapshot_table=update_snapshot_table),update_snapshot_sql=snapshot_col.format(update_snapshot_table=update_snapshot_table),database=dbmap[dbtype]["database"],yesterday=yesterday,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
        print(real_sql)
                df=spark.sql(real_sql).drop("etlindex")
        if colmap.get(dbtype)!=None and colmap[dbtype].get(tableName)!=None :
            for i in colmap[dbtype][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        if colmap["extraColumn"]!=None and colmap["extraColumn"].get(dbtype)!=None and colmap["extraColumn"][dbtype].get(tableName)!=None and len(colmap["extraColumn"][dbtype][tableName])>0:
            colmap["extraColumn"][dbtype][tableName].insert(0,"*")
            df=df.selectExpr(colmap["extraColumn"][dbtype][tableName])
            df=df.withColumn("year_new",F.col("year")).drop("year").withColumnRenamed("year_new","year")
        df.drop("col_x").write.option("path", "s3://rupiahplus-data-warehouse/stream/etl_s3_temp/"+dbtype+"_"+tableName).mode("overwrite").partitionBy("year").format("orc").saveAsTable("etl_s3_temp."+dbtype+"_"+tableName);
        tablepath='s3://rupiahplus-data-warehouse/aliyun/'+dbmap[dbtype]["realTime_path"]+'/'+tableName+"/"
        print(spark.sql("select  * from  "+"etl_s3_temp."+dbtype+"_"+tableName).count())
        spark.sql("select  * from  "+"etl_s3_temp."+dbtype+"_"+tableName).write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
    print("start-----",datetime.now())
    for dbtype in dbmap:
        databasesql="show tables in "+dbmap[dbtype]["increment_database"]
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
    #         for row in tablelist:
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(execute, table,dbtype) for table in tablelist])
            for  future in futures_result[0]:
                print(future.result())
                print("end-----",datetime.now())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start----- 2021-06-09 11:07:43.921562
`banda-etl-s3`.t_auto_review_loan
`banda_stream_etl`.t_auto_review_loan
None
end----- 2021-06-09 11:07:45.614753
None
end----- 2021-06-09 11:07:45.614798
None
end----- 2021-06-09 11:07:45.614820
None
end----- 2021-06-09 11:07:45.614841
None
end----- 2021-06-09 11:07:45.614861
None
end----- 2021-06-09 11:07:45.614881
None
end----- 2021-06-09 11:07:45.614900
None
end----- 2021-06-09 11:07:45.614919
None
end----- 2021-06-09 11:07:45.614938
None
end----- 2021-06-09 11:07:45.614958
None
end----- 2021-06-09 11:07:45.614978
None
end----- 2021-06-09 11:07:45.615006
None
end----- 2021-06-09 11:07:45.615026
None
end----- 2021-06-09 11:07:45.615045
None
end----- 2021-06-09 11:07:45.615065
None
end----- 2021-06-09 11:07:45.615084
None
end----- 2021-06-09 11:07:45.615103
None
end----- 2021-06-09 11:07:45.615123
None
end----- 2021-06-09 11:07:45.615146
None
end----- 2021-06-09 11:07:45.615167
None
end----- 2021-06-09 11:07:45.615191
None
end----- 2021-06-09 11:0

In [1]:
spark.sql("""
with update_snapshot_banda_t_auto_review_loan as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `banda_stream_etl`.`t_auto_review_loan`
         where date(year || '-' || month || '-' || day) <=date('2021-06-09') and date(year || '-' || month || '-' || day) >= date('2021-06-08')
          and etldate <= date_format(now(), 'yyyy-MM-dd hh:00:00.000')
     )
    where row_num = 1
    and id>20925940 and id < 20926000
  )
select  if(update_snapshot_banda_t_auto_review_loan.id is null, l.etldate     , update_snapshot_banda_t_auto_review_loan.etldate)  etldate ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.id     , update_snapshot_banda_t_auto_review_loan.id)  id ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.riskengine_id     , update_snapshot_banda_t_auto_review_loan.riskengine_id)  riskengine_id ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.customer_id     , update_snapshot_banda_t_auto_review_loan.customer_id)  customer_id ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.loan_id     , update_snapshot_banda_t_auto_review_loan.loan_id)  loan_id ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.name     , update_snapshot_banda_t_auto_review_loan.name)  name ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.ktp_x     , update_snapshot_banda_t_auto_review_loan.ktp)  ktp ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.mobile_x     , update_snapshot_banda_t_auto_review_loan.mobile)  mobile ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.success     , update_snapshot_banda_t_auto_review_loan.success)  success ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.error_message     , update_snapshot_banda_t_auto_review_loan.error_message)  error_message ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.result     , update_snapshot_banda_t_auto_review_loan.result)  result ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.reject_message     , update_snapshot_banda_t_auto_review_loan.reject_message)  reject_message ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.create_time     , update_snapshot_banda_t_auto_review_loan.create_time)  create_time ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.finish_time     , update_snapshot_banda_t_auto_review_loan.finish_time)  finish_time ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.duration     , update_snapshot_banda_t_auto_review_loan.duration)  duration ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.real_policy     , update_snapshot_banda_t_auto_review_loan.real_policy)  real_policy ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.switch_manual_reason     , update_snapshot_banda_t_auto_review_loan.switch_manual_reason)  switch_manual_reason ,if(update_snapshot_banda_t_auto_review_loan.id is null, l.raw_data     , update_snapshot_banda_t_auto_review_loan.raw_data)  raw_data , if(update_snapshot_banda_t_auto_review_loan.id is null, l.ktp     , update_snapshot_banda_t_auto_review_loan.ktp ) ktp_x ,  if(update_snapshot_banda_t_auto_review_loan.id is null, l.mobile     , update_snapshot_banda_t_auto_review_loan.mobile  )mobile_x , if(update_snapshot_banda_t_auto_review_loan.id is null, l.year     , update_snapshot_banda_t_auto_review_loan.year)  year 
from `banda-etl-s3`.`t_auto_review_loan` l
left join update_snapshot_banda_t_auto_review_loan
on l.id = update_snapshot_banda_t_auto_review_loan.id
where if(update_snapshot_banda_t_auto_review_loan.kind is not null,update_snapshot_banda_t_auto_review_loan.kind,'') <> 'delete'
and l.id=2091985111111111111
union
select update_snapshot_banda_t_auto_review_loan.etldate ,update_snapshot_banda_t_auto_review_loan.id ,update_snapshot_banda_t_auto_review_loan.riskengine_id ,update_snapshot_banda_t_auto_review_loan.customer_id ,update_snapshot_banda_t_auto_review_loan.loan_id ,update_snapshot_banda_t_auto_review_loan.name ,update_snapshot_banda_t_auto_review_loan.ktp ,update_snapshot_banda_t_auto_review_loan.mobile ,update_snapshot_banda_t_auto_review_loan.success ,update_snapshot_banda_t_auto_review_loan.error_message ,update_snapshot_banda_t_auto_review_loan.result ,update_snapshot_banda_t_auto_review_loan.reject_message ,update_snapshot_banda_t_auto_review_loan.create_time ,update_snapshot_banda_t_auto_review_loan.finish_time ,update_snapshot_banda_t_auto_review_loan.duration ,update_snapshot_banda_t_auto_review_loan.real_policy ,update_snapshot_banda_t_auto_review_loan.switch_manual_reason ,update_snapshot_banda_t_auto_review_loan.raw_data ,update_snapshot_banda_t_auto_review_loan.ktp ktp_x , update_snapshot_banda_t_auto_review_loan.mobile mobile_x ,update_snapshot_banda_t_auto_review_loan.year from (
  select *
  from `banda_stream_etl`.`t_auto_review_loan`
  where
    date(year || '-' || month || '-' || day) <=date('2021-06-09') and date(year || '-' || month || '-' || day) >= date('2021-06-08')
    and kind = 'insert'
  ) new
left join update_snapshot_banda_t_auto_review_loan
on new.id = update_snapshot_banda_t_auto_review_loan.id where if(update_snapshot_banda_t_auto_review_loan.kind is not null,update_snapshot_banda_t_auto_review_loan.kind,'') <> 'delete' 
and new.id>20925940  and new.id< 20926000
""").show(100)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1623207252752_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+--------------------+-----------+--------+--------------------+----------------+-------------+-------+-------------+------+--------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+----------------+-------------+----+
|             etldate|      id|       riskengine_id|customer_id| loan_id|                name|             ktp|       mobile|success|error_message|result|reject_message|         create_time|         finish_time|duration|         real_policy|switch_manual_reason|            raw_data|           ktp_x|     mobile_x|year|
+--------------------+--------+--------------------+-----------+--------+--------------------+----------------+-------------+-------+-------------+------+--------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+----------------+-------------+----+
|2021-06-09 07:56:...|20925995|ce92deb5-

In [1]:
spark.sql("""
select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `banda_stream_etl`.`t_auto_review_loan`
         where date(year || '-' || month || '-' || day) <=date('2021-06-09') and date(year || '-' || month || '-' || day) >= date('2021-06-08')
          and etldate <= date_format(now(), 'yyyy-MM-dd hh:00:00.000')
     )
    where row_num = 1
    and id=20925945
""").show()





VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1623207252752_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+--------+--------+--------------------+-----------+--------+------------+----------------+------------+-------+-------------+------+--------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+----+-----+---+-------+
|  kind|             etldate|etlindex|      id|       riskengine_id|customer_id| loan_id|        name|             ktp|      mobile|success|error_message|result|reject_message|         create_time|         finish_time|duration|         real_policy|switch_manual_reason|            raw_data|year|month|day|row_num|
+------+--------------------+--------+--------+--------------------+-----------+--------+------------+----------------+------------+-------+-------------+------+--------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+----+-----+---+-------+
|update|2021-06-09 07:55:...|       0|20925945|288c992b-a8

In [4]:
spark.sql("""
select       and to_timestamp(etldate)< to_timestamp('2021-06-09 05:00:00.000')
""").show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------+
|to_timestamp('2021-06-09 04:59:19.089')|
+---------------------------------------+
|2021-06-09 04:59:19.089                |
+---------------------------------------+

In [1]:
spark.sql("""Select  now() """).show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1623207252752_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               now()|
+--------------------+
|2021-06-09 10:56:...|
+--------------------+

In [3]:
df=spark.sql("""SELECT 1  """)
df.drop("col_x").write.option("path", "s3://rupiahplus-data-warehouse/stream/etl_s3_temp/aaa_bbb").mode("overwrite").format("orc").saveAsTable("etl_s3_temp.aaa_bbb");

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.executor.instances", "43") \
    .enableHiveSupport()\
    .getOrCreate()
spark.sql("""select 1 """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
|  1|
+---+
|  1|
+---+